In [1]:
import numpy as np
import tifffile as tiff
from PIL import Image
import os
import re
import math

In [2]:
output_dir = "../output/Minerva_story/Merged/"
output_temporary = "../output/Minerva_story/Merged/temp"
if not os.path.isdir(output_temporary):
    os.mkdir(output_temporary)
input_image_dir = "../output/input/"
input_segmentation_dir = "../output/segmentation/"


In [3]:
sample_list = dict(HD = {"ROI-13", "ROI-28"},
                  AD = {"ROI-09", "ROI-15"},
                  PS = {"ROI-04", "ROI-16"},
                  LP = {"ROI-08", "ROI-24"},
                  MF = {"ROI-06", "ROI-23", "ROI-12", "ROI-27"}, 
                  SS = {"ROI-05", "ROI-11", "ROI-17", "ROI-22"}
                  )
all_samp = sorted({x for v in sample_list.values() for x in v})

In [4]:
markers = ("DAPI", "Cytokeratin", "Actin", "Podoplanin", "CD3", "CD4", "CD8a", "FoxP3",
           "CD14", "HLADR", "CD68", "CD11b", "CD123", "CD79a", "CD57", "FcepsilonRIalpha", "CollagenI",
           "Bcl2", "CD2", "CD5", "Galectin3", "Galectin9", "HLAABC", "CLA", "CD195", "CD196", "CD1c"
          )

In [25]:
##########################################################################
# Padding and resizing
##########################################################################

In [5]:
def load_and_pad_image(image_path, target_size=(8000, 7000)):
    image = Image.open(image_path)
    image_array = np.array(image)
    
    # Create a black (zero) image of the target size
    padded_image = np.zeros(target_size, dtype=image_array.dtype)
    
    # Get the dimensions of the original image
    original_height, original_width = image_array.shape
    
    # Place the original image at the top-left corner of the padded image
    padded_image[:original_height, :original_width] = image_array
    
    return padded_image

In [27]:
## Markers

In [56]:
options = {
        'compression': 'zlib'  # Apply compression
    }

for marker in markers:
    for sample in all_samp:
        
        image_path = os.path.join(input_image_dir, sample, marker + ".tiff")
        padded_image = load_and_pad_image(image_path)
        padded_image = (padded_image / (256)).astype(np.uint8)

        if not os.path.isdir(os.path.join(output_temporary, sample)):
            os.mkdir(os.path.join(output_temporary, sample))
        
        tiff.imwrite(os.path.join(output_temporary, sample, marker + ".tiff"), padded_image, **options)

In [11]:
all_samp

['ROI-04',
 'ROI-05',
 'ROI-06',
 'ROI-08',
 'ROI-09',
 'ROI-11',
 'ROI-12',
 'ROI-13',
 'ROI-15',
 'ROI-16',
 'ROI-17',
 'ROI-22',
 'ROI-23',
 'ROI-24',
 'ROI-27',
 'ROI-28']

In [ ]:
## Segmentation + convert to uint32

In [6]:
for sample in all_samp:        

    num = 1000000 * np.uint32(re.sub("ROI-0|ROI-", "", sample))

    image_path = os.path.join(input_segmentation_dir, sample + "_whole_cell.tiff")
    padded_image = load_and_pad_image(image_path)
    padded_image[padded_image > 0] = padded_image[padded_image > 0] + num
    padded_image = padded_image.astype(np.uint32)
    
    tiff.imwrite(os.path.join(output_temporary, sample,  sample + "_whole_cell.tiff"), padded_image)

In [32]:
##########################################################################
# Concatenating images on a grid
##########################################################################

In [7]:
def concatenate_images(images, grid_shape=(4, 4), image_size=(8000, 7000)):
    # Create an empty array for the concatenated grid
    concatenated_image = np.zeros((grid_shape[0] * image_size[0], grid_shape[1] * image_size[1]), dtype=images[0].dtype)
    
    for idx, img in enumerate(images):
        row = idx // grid_shape[1]
        col = idx % grid_shape[1]
        concatenated_image[row*image_size[0]:(row+1)*image_size[0], col*image_size[1]:(col+1)*image_size[1]] = img
    
    return concatenated_image



In [8]:
def save_tiff(image, output_path, axes='CYX', channel_names=None, compression='zlib'):
    options = {
        'metadata': {'axes': axes, 'Channel': [{'Name': name} for name in channel_names]},
        'compression': compression
    }
    tiff.imwrite(output_path, image, **options)

In [46]:
## Markers

In [74]:
options = {
        'compression': 'zlib'  # Apply compression
    }
 
all_concatenated_images = []
for marker in markers:
    images = []
    for sample in all_samp:
        image_path = os.path.join(output_temporary, sample, marker + ".tiff")
        image = tiff.imread(image_path)
        images.append(image)
    
    concatenated_image = concatenate_images(images)
    # max_value = concatenated_image.max()
    # max_value = np.quantile(a=concatenated_image, q = 0.99)
    # concatenated_image[concatenated_image > max_value] = max_value
    # concatenated_image = concatenated_image * (255 / max_value)
    concatenated_image = concatenated_image.astype(np.uint8)

   
    tiff.imwrite(os.path.join(output_dir,  marker + ".tif"), concatenated_image, **options)
    

In [13]:
all_samp

['ROI-04',
 'ROI-05',
 'ROI-06',
 'ROI-08',
 'ROI-09',
 'ROI-11',
 'ROI-12',
 'ROI-13',
 'ROI-15',
 'ROI-16',
 'ROI-17',
 'ROI-22',
 'ROI-23',
 'ROI-24',
 'ROI-27',
 'ROI-28']

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 156, 157, 158,
       159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
       172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 18

In [12]:
all_samp

['ROI-04',
 'ROI-05',
 'ROI-06',
 'ROI-08',
 'ROI-09',
 'ROI-11',
 'ROI-12',
 'ROI-13',
 'ROI-15',
 'ROI-16',
 'ROI-17',
 'ROI-22',
 'ROI-23',
 'ROI-24',
 'ROI-27',
 'ROI-28']

In [33]:
## Segmentation

In [9]:
options = {
        'compression': 'zlib'  # Apply compression
    }

segmentations = []
for sample in all_samp:
    image_path = os.path.join(output_temporary, sample, sample + "_whole_cell.tiff")
    image = tiff.imread(image_path)
    segmentations.append(image)
    
concatenated_segmentation = concatenate_images(segmentations)
tiff.imwrite(os.path.join(output_dir, "whole_cell.tif"), concatenated_segmentation, **options)


In [ ]:
save_as_ome_tiff(concatenated_segmentation, os.path.join(output_dir, "whole_cell.ome.tif"))

In [10]:
def save_as_ome_tiff(image, output_path, axes='CYX', channel_names=None, compression='zlib'):
    options = {
        'metadata': {'axes': axes, 'Channel': [{'Name': name} for name in channel_names]},
        'compression': compression
    }
    tiff.imwrite(output_path, image, **options)


In [31]:
big_tiff = tiff.imread("../output/Snapshots/scheme_minerva_2.tiff")



FileNotFoundError: [Errno 2] No such file or directory: '/mnt/RECHERCHE/GUENOVA_LAB/Project_Multiplex_Phenotyping/Miltenyi/Muliplex_Imaging_Pipeline/output/Snapshots/scheme_minerva_2.tiff'

In [19]:
big_tiff.shape

(19842, 23605, 4)

In [21]:
 # Create a black (zero) image of the target size
padded_image = np.zeros((32000,28000,4), dtype=big_tiff.dtype)



In [22]:
padded_image.shape

(32000, 28000, 4)

In [24]:
# Get the dimensions of the original image
original_height, original_width, original_channels = big_tiff.shape


In [25]:
# Place the original image at the top-left corner of the padded image
padded_image[:original_height, :original_width, :original_channels] = big_tiff


In [29]:
padded_image.shape

(32000, 28000, 4)

In [28]:
big_tiff.shape

(19842, 23605, 4)

In [30]:
save_as_ome_tiff(padded_image, "../output/Minerva_story/scheme_minerva.ome.tif", axes = "YXC", channel_names=["Red", "Green", "Blue", "Alpha"])

In [28]:
def resize_image(image, target_size):
    return image.resize(target_size, Image.NEAREST)

def resize_multi_channel_tiff(input_path, output_path, target_size):
    # Load the multi-channel TIFF image
    with tiff.TiffFile(input_path) as tif:
        image = tif.asarray()
    
    # Verify the number of channels; ensure it matches the expected dimensions
    if image.shape[2] != 4:
        raise ValueError("Expected a 4-channel image.")
    
    # Split the multi-channel image into separate channels
    channels = [Image.fromarray(image[:, :, i]) for i in range(4)]
    
    # Resize each channel
    resized_channels = [resize_image(channel, target_size) for channel in channels]
    
    # Convert resized channels back to numpy arrays
    resized_channels = [np.array(channel) for channel in resized_channels]
    
    # Stack the resized channels along the third axis to recreate the multi-channel image
    resized_image = np.stack(resized_channels, axis=2)

    # Save the resized multi-channel image as a TIFF file
    channel_names=["Red", "Green", "Blue", "Alpha"]
 
    
    for i in range(3):
        
        options = {
            'compression': "zlib",
            'metadata': {
                'axes': "YX",
                'Channel': channel_names[i],
            }
        }
        tiff.imwrite("../output/Snapshots/" + channel_names[i] + ".tif", resized_image[:,:,i], **options)
    
    

In [25]:
resized_image[:,:,0].shape

(28000, 32000)

In [29]:
input_path = '/home/localadmin/scheme_minerva_2.tiff'
output_path = '../output/Snapshots/scheme_minerva_3_resized.ome.tif'
target_size = (32000, 28000)

In [30]:
resize_multi_channel_tiff(input_path, output_path, target_size)

In [16]:
image = tiff.imread("../output/Snapshots/Scheme.ome.tif")

<tifffile.TiffFile 'Scheme.ome.tif'> ImageJ series metadata invalid or corrupted file


In [20]:
image

array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]]], dtype=uint8)